## 9주차 recsys basic 과제: Q1, Q2에있는 빈칸을 채워주세요

## Movielens data 탐방하기

In [69]:

import argparse

import torch

#다양한 hyperparameter를 조절하기 위한 argparser
argparser = argparse.ArgumentParser()
argparser.add_argument('--latent_dim', type=int, default=10)
argparser.add_argument('--batch_size', type=int, default=500)
args = argparser.parse_args("")

In [70]:
import pandas as pd
import numpy as np
# 이건 토치용입니다.
from torch.utils.data import Dataset, DataLoader
import torch

class MlDataLoader(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


## 무비렌즈 데이터 불러주는  class: ml100k
ML100k 클래스에서 data_path만 여러분이 u.data를 불러올 수 있는 경로로 바꿔주세요!!

In [71]:
import pandas as pd
# traintest split
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
#from src.preprocess.mldataloader import MlDataLoader


class ML100k():

    def __init__(self,args) -> None:
        self.args=args
        self.call_data(args)
        pass

    def call_data(self, args) -> None:
        self.args = args
        data_path="/content/sample_data/u.data" #이부분을 다운받은 데이터를 코랩에서 실행할 수 잇도록 바꿔주세요

        self.data= pd.read_csv(data_path, sep='\t', header=None, names=['user', 'item', 'rating', 'timestamp'])
        self.num_users = self.data.user.nunique()
        self.num_items = self.data.item.nunique()
        self.num_ratings = len(self.data)

        # user, item index를 0부터 시작하도록 변경-. nn.Embedding에서 index가 0부터 시작하므로 1을 빼줍니다.
        self.data.user = self.data.user -1
        self.data.item = self.data.item -1


        self.x = self.data[['user', 'item']].values
        self.y= self.data.rating.values

        # split data into train and test
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.x, self.y, test_size=0.2, random_state=42)




    def get_dataframe(self):
        # sort dataframe by user
        return self.data[['user', 'item','rating']]


    def load_torch_data(self):

        # convert x to integer, y to floattenser
        self.x_train = torch.tensor(self.x_train, dtype=torch.long)
        self.y_train = torch.tensor(self.y_train, dtype=torch.float)
        self.x_test = torch.tensor(self.x_test, dtype=torch.long)
        self.y_test = torch.tensor(self.y_test, dtype=torch.float)


        train_dataset = MlDataLoader(self.x_train, self.y_train)
        test_dataset = MlDataLoader(self.x_test, self.y_test)

        train_loader = DataLoader(train_dataset, batch_size=self.args.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=self.args.batch_size, shuffle=False)

        return train_loader, test_loader


In [72]:
ml100k=ML100k(args)
dataframe=ml100k.get_dataframe()

In [73]:
dataframe# 아래와 같이 user, item, rating으로 이루어진 데이터프레임이 나오게 됩니다.

,user,item,rating
0,195,241,3
1,185,301,3
2,21,376,1
3,243,50,2
4,165,345,1
...,...,...,...
99995,879,475,3
99996,715,203,5
99997,275,1089,1
99998,12,224,2


##  Q1 : 해당 데이터 프레임을 통해서 각각의 유저가 item들의 점수를 어떻게 매겼는지를 나타내는 user-item matrix(rating matrix)를 만들어 주세요!!
이 때 유저가 시청하지 않은 데이터는 -1로 처리해주세요!

In [74]:
rating_matrix = np.full((dataframe['user'].max() + 1, dataframe['item'].max() + 1), -1)

rating_matrix[dataframe['user'], dataframe['item']] = dataframe['rating']
rating_matrix

array([[ 5,  3,  4, ..., -1, -1, -1],
       [ 4, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [ 5, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1,  5, -1, ..., -1, -1, -1]])

## -1로 채워진 부분들을 채워주기 위해서 matrix factorization을 해봅시다
이때 torch를 활용해서 sgd기반의 방법으로 user embedding과 item embedding을 훈련시켜 보겠습니다.


In [75]:
# 위에 ml100k 클래스에서 torch dataloader를 불러오는 함수를 호출하여 데이터로더 형태의 train_loader, test_loader를 불러옵니다.
train_loader, test_loader=ml100k.load_torch_data()



이제 Matrix factorization model을 구현해봅시다.

In [76]:
import torch
import torch.nn as nn

class MF(nn.Module):

    def __init__(self, args ,num_users,num_items) -> None:
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.latent_dim = args.latent_dim
        self.user_embedding = nn.Embedding(self.num_users, self.latent_dim)
        self.item_embedding = nn.Embedding(self.num_items, self.latent_dim)
        #self.dropout = nn.Dropout(p=args.dropout)


    def forward(self, x) -> None:
        # x: (batch_size, 2) : index 0:user, index 1:item
        user = x[:, 0]
        item = x[:, 1]
        user = self.user_embedding(user)
        #user shape: (batch_size, latent_dim) 우리가 원하는 유저가 표현된 latent_dim 차원의 벡터
        item = self.item_embedding(item)
        #item shape: (batch_size, latent_dim) 우리가 원하는 아이템이 표현된 latent_dim 차원의 벡터

        out = torch.sum(user * item, dim=1) # 내적을 통해서 구하기,

        return out


여기서 잠깐 EMbedding이 어떻게 동작하는지 확인한번만 해봅시다

In [77]:
uemb_temp=nn.Embedding(ml100k.num_users, args.latent_dim)
iembe_temp=nn.Embedding(ml100k.num_items, args.latent_dim)

In [78]:
print(uemb_temp.weight.shape)
print(iembe_temp.weight.shape)

torch.Size([943, 10])
torch.Size([1682, 10])


In [79]:
uemb_temp.weight

Parameter containing:
tensor([[ 1.6092, -0.2925,  0.3341,  ...,  0.6207, -0.2889,  0.4803],
        [-0.7693, -0.3107,  1.1864,  ...,  0.6789, -1.2579,  1.8524],
        [-1.2727,  0.4376, -0.4932,  ..., -0.5677,  1.3732,  0.3531],
        ...,
        [ 0.4895, -0.9799,  2.1518,  ..., -0.4165,  0.6835,  0.8495],
        [ 0.0333,  0.3687,  0.1292,  ..., -0.0571,  0.8550,  0.2734],
        [ 0.0932,  3.4748,  1.9297,  ..., -0.9134,  1.4667, -0.4475]],
       requires_grad=True)

In [80]:
iembe_temp.weight

Parameter containing:
tensor([[-1.0616, -0.6290,  0.8885,  ..., -1.1581,  0.6008, -0.1896],
        [-0.9930, -1.5063,  0.6419,  ..., -1.5270, -0.7092, -0.1773],
        [ 0.2015,  1.7572,  1.7929,  ..., -0.9768,  0.4656,  0.1508],
        ...,
        [-0.9637, -0.1488,  2.3127,  ..., -1.7764, -1.0335, -0.3968],
        [-1.6026,  0.7100,  1.5547,  ..., -1.5642, -0.4212,  1.1615],
        [ 0.6856, -0.2166, -0.6354,  ..., -0.1072, -0.5295,  1.0799]],
       requires_grad=True)

즉 nn.Embedding을 통해서 유저, 아이템에 대한 embedding matrix를 저장하고 훈련시킬 수 잇는 네트워크라고 보시면 됩니다.(자연어에서 사용되는 nn.embedding과 같습니다.`)

그럼 이제 모델을 훈련시켜봅시다. 엄밀히 훈련시키려면 regularization,earlystopping등의 장치가 추가적으로 필요하지만, 돌아가는거만 확인하기 위해서 최대한 간단한 형태로 구현했습니다.

In [81]:
import torch
class Trainer():

    def __init__(self, model, optimizer, criterion, device):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device


    def train(self, data, test_data, epochs):
        for epoch in range(epochs):
            for (x,y) in data:
                x = x.to(self.device)
                y = y.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(x)
                loss = self.criterion(output, y)
                loss.backward()
                self.optimizer.step()
                # print test loss
            test_loss = 0
            with torch.no_grad():
                for (x,y) in test_data:
                    x = x.to(self.device)
                    y = y.to(self.device)
                    output = self.model(x)
                    test_loss += self.criterion(output, y)

                #normalize test loss
            test_loss /= len(test_data)
            print(f"Epoch {epoch} train loss: {loss.item()} test loss: {test_loss.item()}")

        print("Training finished")




In [82]:
# model, optimizer, criterion, device를 정의합니다.
model = MF(args, ml100k.num_users, ml100k.num_items)
optimizer = torch.optim.Adam(model.parameters(),lr=0.05)
criterion = nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion.to(device)

# trainer를 정의합니다.
trainer = Trainer(model, optimizer, criterion, device)
trainer.train(train_loader, test_loader, 10)

Epoch 0 train loss: 2.051802635192871 test loss: 2.033735513687134
Epoch 1 train loss: 1.1321650743484497 test loss: 1.2944684028625488
Epoch 2 train loss: 0.8841235637664795 test loss: 1.223599910736084
Epoch 3 train loss: 0.8270114064216614 test loss: 1.1969594955444336
Epoch 4 train loss: 0.9763408899307251 test loss: 1.2240242958068848
Epoch 5 train loss: 0.8443388938903809 test loss: 1.2199833393096924
Epoch 6 train loss: 1.0285223722457886 test loss: 1.237939715385437
Epoch 7 train loss: 0.8599250912666321 test loss: 1.2642741203308105
Epoch 8 train loss: 0.979418933391571 test loss: 1.285125970840454
Epoch 9 train loss: 0.8273536562919617 test loss: 1.2779037952423096
Training finished


이제 아까 빈칸이었던 user-item matrix를 채워봅시다!




In [83]:
user_item_matrix=dataframe.pivot(index='user', columns='item', values='rating').fillna(-1) #pandas로 하면 너무느려져서 numpy로 바구고 진행합시다
user_item_matrix=user_item_matrix.to_numpy()

In [84]:
model=model.cpu() #모델을 cpu로 바꿔주고
# 훈련된 embedding을 numpy로 바꿔줍니다.
np_userembedding=model.user_embedding.weight.detach().numpy()
np_itemembedding=model.item_embedding.weight.detach().numpy()

In [85]:
user_item_matrix

array([[ 5.,  3.,  4., ..., -1., -1., -1.],
       [ 4., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [ 5., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1.,  5., -1., ..., -1., -1., -1.]])

## Q2. 아래 코드를 완성해주세요!

In [88]:
# 아래 코드를 완성해주세요
for i in range(ml100k.num_users):
    for j in range(ml100k.num_items):
        if user_item_matrix[i,j]==-1:
            user_item_matrix[i,j]= np.dot(np_userembedding[i], np_itemembedding[j]) # 물음표를 채워주세요! , HInt  : 내적

## 결과

In [89]:
user_item_matrix

array([[ 5.        ,  3.        ,  4.        , ...,  1.03263474,
         2.61526203,  1.48521328],
       [ 4.        ,  3.96665692,  3.61533237, ...,  0.34918749,
         3.74052668,  1.50529671],
       [ 4.01101589,  1.05548322,  4.06618261, ...,  0.58951843,
         7.57634878,  0.56986988],
       ...,
       [ 5.        ,  4.70909214,  4.33556032, ...,  3.03859639,
         1.08628368,  3.50494242],
       [ 4.6394043 ,  3.54699659,  4.25739908, ..., -0.78449368,
         4.16478205,  0.18332827],
       [ 2.977283  ,  5.        ,  2.67085958, ...,  1.07822251,
         2.22594523,  0.45104298]])

### 추가로....
결과를 보면 범위가 -3 에서  +5보다 큰 값도 나왔는데, 일반적으로는 0.5~5 사이로 normalization을 시켜주는 것 같습니다. 이건 생략하겠습니다 ㅎㅎ..